In [ ]:
import time
import pandas as pd
import numpy as np
import os
from datetime import date
import time
today = date.today()
path = os.path.dirname(os.getcwd())
print(f'📂 Current working directory: {path}')
print(f'💚 Today is {today}')

💚 Today is 2025-03-11


# Try: Get Citation Networks Datasets

In [2]:
df = pd.read_csv(f'{path}/data_abstracts/abstracts_2025-03-04.csv')
len(df)

11023

In [3]:
print(df['sort_gpt_1'].value_counts())
print(df['sort_gpt4o_1'].value_counts())
# A lot of false's

sort_gpt_1
false          9004
empirical      1381
theoretical     352
both            212
Name: count, dtype: int64
sort_gpt4o_1
false          9370
empirical      1138
theoretical     295
both            147
Name: count, dtype: int64


In [4]:
print(df['language'].value_counts())
print(len(df[(df['abstract'].isnull() == True) & (df['abstract_sm'].isnull() == True)]))

language
en    11023
Name: count, dtype: int64
0


In [5]:
# Let's first start with a smaller subset of the data which has higher chance 
# of containing papers actually stuyding wealth mobility.
data = df[(df['sort_gpt_1'] != 'false') | (df['sort_gpt4o_1'] != 'false')]
len(data) #2169 is better... 

2169

In [6]:
from pyalex import Works
from itertools import chain

In [7]:
def get_citations(id):
    short_id = id.split('openalex.org/')[-1]
    w = Works()[short_id]
    # references = w['referenced_works']
    # len_ref = len(references) # Actually don't need references
    ref_count = w['referenced_works_count']
    query = Works().filter(cites=short_id).select('id')
    cited = []
    for record in chain(*query.paginate(per_page=200)):
        cited.append(record)
    cited_by = [item['id'] for item in cited]
    len_cited_by = len(cited_by)
    cited_by_count = w['cited_by_count']
    return cited_by, len_cited_by, ref_count, cited_by_count

In [9]:
for id in data['id'].unique():
    start = time.time()
    print(f'📚 Getting citations for {id}')
    cited_by, len_cited_by, ref_count, cited_by_count = get_citations(id)
    cited_by = ', '.join(cited_by)
    data.loc[data['id'] == id, 'cited_by'] = cited_by # This only give max.25 cited by papers. 
    data.loc[data['id'] == id, 'len_cited_by'] = len_cited_by
    data.loc[data['id'] == id, 'ref_count'] = ref_count
    data.loc[data['id'] == id, 'cited_by_count'] = cited_by_count
print('✅ Done! The process time is:', time.time()-start)

📚 Getting citations for https://openalex.org/W1994619981
📚 Getting citations for https://openalex.org/W2564827629
📚 Getting citations for https://openalex.org/W2015903886
📚 Getting citations for https://openalex.org/W2117407660
📚 Getting citations for https://openalex.org/W3123373494
📚 Getting citations for https://openalex.org/W1515121029
📚 Getting citations for https://openalex.org/W2008496439
📚 Getting citations for https://openalex.org/W3123152915
📚 Getting citations for https://openalex.org/W2810621440
📚 Getting citations for https://openalex.org/W2138126113
📚 Getting citations for https://openalex.org/W2081092446
📚 Getting citations for https://openalex.org/W2161753386
📚 Getting citations for https://openalex.org/W2125190089
📚 Getting citations for https://openalex.org/W2046200558
📚 Getting citations for https://openalex.org/W2021744740
📚 Getting citations for https://openalex.org/W2155278075
📚 Getting citations for https://openalex.org/W3125010010
📚 Getting citations for https:/

In [10]:
pd.set_option('display.max_colwidth', None)  # Ensures full text display
data.nlargest(30, 'cited_by_count')[['id','title','len_cited_by', 'cited_by_count', 'sort_gpt_1', 'sort_gpt4o_1', 'oa_link']]

,id,title,len_cited_by,cited_by_count,sort_gpt_1,sort_gpt4o_1,oa_link
8702,https://openalex.org/W2105584013,Where is the land of Opportunity? The Geography of Intergenerational Mobility in the United States *,2346.0,2341.0,empirical,empirical,https://academic.oup.com/qje/article-pdf/129/4/1553/30631636/qju022.pdf
8428,https://openalex.org/W2022256544,An Equilibrium Theory of the Distribution of Income and Intergenerational Mobility,2092.0,2152.0,theoretical,theoretical,NaN
8744,https://openalex.org/W2122216841,"Income Inequality, Equality of Opportunity, and Intergenerational Mobility",1522.0,1529.0,empirical,empirical,https://doi.org/10.1257/jep.27.3.79
157,https://openalex.org/W2098947378,Social Mobility and Redistributive Politics,1331.0,1362.0,theoretical,theoretical,https://dspace.mit.edu/bitstream/1721.1/64248/1/socialmobilityre00pike.pdf
2,https://openalex.org/W2564827629,The fading American dream: Trends in absolute income mobility since 1940,702.0,703.0,empirical,empirical,https://apps.bea.gov/scb/pdf/2007/02%20February/0207_history_article.pdf
8468,https://openalex.org/W2037106172,Equity and Efficiency in Human Capital Investment: The Local Connection,677.0,696.0,theoretical,theoretical,NaN
328,https://openalex.org/W2120406225,Neighborhood Effects on the Long-Term Well-Being of Low-Income Adults,672.0,675.0,empirical,false,https://europepmc.org/articles/pmc3491569?pdf=render
363,https://openalex.org/W2100746011,The Economics of Human Development and Social Mobility,667.0,666.0,theoretical,both,http://www.nber.org/papers/w19925.pdf
8739,https://openalex.org/W2119587652,Cross-Country Differences in Intergenerational Earnings Mobility,642.0,660.0,both,both,https://www.aeaweb.org/articles/pdf/doi/10.1257/089533002760278712
8664,https://openalex.org/W2091916425,Fortunate Sons: New Estimates of Intergenerational Mobility in the United States Using Social Security Earnings Data,643.0,654.0,empirical,empirical,NaN


In [10]:
data.to_csv(f'{path}/data_abstracts/abstracts_citations_2025-03-05.csv', index=False)

In [22]:
len(data)

2169

# O3-mini for Categorizations Q1-4

In [41]:
from openai import OpenAI
from dotenv import load_dotenv
# OpenAI
load_dotenv() 
openai_API_KEY = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key=openai_API_KEY)

In [42]:
# gpt_model = "gpt-4o-2024-08-06"
gpt_model = "o3-mini"

In [43]:
from pydantic import BaseModel

class GetInfo(BaseModel):
    Q1: str
    Q1_1: str
    Q2: str
    Q2_1: str
    Q2_2: str
    Q3: str
    Q4: str

In [49]:
df = pd.read_csv(f'{path}/data_abstracts/abstracts_citations_gpt_extra_2025-03-11.csv')
df = df[(df['abstract'].isna() == True) & (df['abstract_sm'].isna() == False)]
len(df) #68

40

In [51]:
df[['abstract', 'abstract_sm']]

,abstract,abstract_sm
4,NaN,We examine the mobility of individuals in the ...
17,NaN,to the social mobility chances of youngsters o...
18,NaN,We analyse intergenerational persistence in in...
203,NaN,This paper examines a dynamic aspect of the di...
218,NaN,The article directs attention to the structuri...
224,NaN,This paper presents a systematic comparison of...
268,NaN,The traditional method of estimating intergene...
271,NaN,This article gives various estimates of interg...
272,NaN,Existing studies on intergenerational income m...
273,NaN,This paper analyzes theoretically and empirica...


In [52]:
import logging
import json

# Enable logging
logging.basicConfig(level=logging.INFO)

results = []

for row in df.itertuples(index=False): 
    start_time = time.time()
    
    logging.info(f'📚 Getting info for {row.id}')
    
    completion = openai_client.beta.chat.completions.parse(
            model= gpt_model,
            messages=[
                { 
                "role": "system",
                "content": "Extract relevant data."
                
                },
                {
                    "role": "user", 
                    "content":f"""
                            Consider this academic paper in the Social Sciences domain: 
                            - Title: {row.title}
                            - Year: {row.year}
                            - Abstract: {row.abstract_sm}
                            - DOI: {row.doi}

                            Use this information to answer the following questions. You can try to access the full paper using the DOI provided.:
                            Q1. Does the paper discuss inter/multigenerational wealth/income/earning mobility? Yes or No.
                            Q1_1. If 'yes' in Q1, give me a sentence or two about what the paper says about inter/multigenerational wealth/income/earning mobility. If 'no' in Q1, what is the main focus of the paper?
                            Q2. If 'yes' in Q1, does this paper mention some measures to study inter/multigenerational wealth/income/earning mobility? Yes or No. 
                            Q2_1. If 'yes' in Q2, what are the measures mentioned in the paper? If 'no' in Q2, return 'N/A'.
                            Q2_2. If 'no' in Q2, give me a reason. If 'yes' in Q2, return 'N/A'.
                            Q3. If 'yes' in Q2, does this paper empirically study inter/multigenerational wealth/income/earning mobility by applying the measures to some real data? Yes or No. 
                            Q4. If 'No' in Q3, does this paper theoretically study the measures of inter/multigenerational wealth/income/earning mobility? Yes or No. 

                            Answer with 'Yes' or 'No', or if irrelevant, 'N/A', for each main questions (Q1,Q2,Q3,Q4). For subquestions (Q1_1, Q2_1, Q2_2), provide a brief explanation. Don't hallucinate. 
                            """
                }
            ],
            response_format=GetInfo,
            # max_tokens=1000,
            # temperature=0.0 # Not for omni-3 model
        )
    answers = completion.choices[0].message.content # or .parse?
    results.append({'id': row.id, 'Q1': answers})
    
logging.info('✅ Done! The process time is:', time.time()-start_time)

with open(f'{path}/data_abstracts/gpt_cat_sm_completeones.json', 'w') as f:
    json.dump(results, f)

INFO:root:📚 Getting info for https://openalex.org/W2027230797
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:📚 Getting info for https://openalex.org/W2027978922
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:📚 Getting info for https://openalex.org/W2912277864
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:📚 Getting info for https://openalex.org/W1575856855
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:📚 Getting info for https://openalex.org/W1763036350
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:📚 Getting info for https://openalex.org/W1856135383
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:📚 Getting info for https://openalex.org/W2009388710
INFO:httpx:HTTP Requ

In [53]:
res_df = pd.DataFrame(results)

In [54]:
res_df

,id,Q1
0,https://openalex.org/W2027230797,"{\n ""Q1"": ""No"",\n ""Q1_1"": ""The paper focuses..."
1,https://openalex.org/W2027978922,"{\n ""Q1"": ""No"",\n ""Q1_1"": ""The paper primari..."
2,https://openalex.org/W2912277864,"{\n ""Q1"": ""Yes"",\n ""Q1_1"": ""The paper analyz..."
3,https://openalex.org/W1575856855,"{\n ""Q1"": ""No"",\n ""Q1_1"": ""The paper focuses..."
4,https://openalex.org/W1763036350,"{\n ""Q1"": ""Yes"",\n ""Q1_1"": ""The paper examin..."
5,https://openalex.org/W1856135383,"{\n ""Q1"": ""No"",\n ""Q1_1"": ""The paper instead..."
6,https://openalex.org/W2009388710,"{\n ""Q1"": ""Yes"",\n ""Q1_1"": ""The paper invest..."
7,https://openalex.org/W2009897266,"{\n ""Q1"": ""Yes"",\n ""Q1_1"": ""The paper examin..."
8,https://openalex.org/W2010545686,"{\n ""Q1"": ""Yes"",\n ""Q1_1"": ""The paper examin..."
9,https://openalex.org/W2011405670,"{\n ""Q1"": ""Yes"",\n ""Q1_1"": ""The paper examin..."


In [55]:
import json
# Parse the JSON strings in the 'Q1' column
res_df[['Q1', 'Q1_1', 'Q2', 'Q2_1', 'Q2_2', 'Q3', 'Q4']] = res_df['Q1'].apply(lambda x: pd.Series(json.loads(x)))

In [56]:
res_df.to_csv(f'{path}/data_abstracts/gpt_cat_sm_completeones.csv', index=False)

# To-do:
- Check whether the results match with the results of previous sort_gpt
- if correspond, gpt o3 mini to get the names of the models they used. 

In [109]:
df = pd.read_csv(f'{path}/data_abstracts/abstracts_citations_2025-03-05.csv')
len(df)

2169

In [ ]:
print(res_df['Q1'].value_counts() , df['sort_gpt4o_1'].value_counts())
# There are more 'No' according to Q1 than 'false' by Sort_GPT4o_1.
# 1 N/A in res_df --> Due to no abstract available in OpenAlex.(id: https://openalex.org/W2071650748) 
# We only worked with OpenAlex Abstracts.

Q1
No     1146
Yes    1022
N/A       1
Name: count, dtype: int64 sort_gpt4o_1
empirical      1138
false           516
theoretical     295
both            147
Name: count, dtype: int64


In [114]:
merged_df = pd.merge(df, res_df, on='id', how='inner')
len(merged_df)

2169

In [ ]:
merged_df[(merged_df['Q1'] == 'No') & (merged_df['sort_gpt4o_1'] != 'false')].Q1_1.value_counts() 
# Q1 seems more accurate than sort_gpt4o_1 since the later didn't include the word 'inter/multigenerational' in the prompt and it was rather a general question.

Q1_1
The paper focuses on life cycle earnings dynamics, using an econometric approach to model earnings mobility and poverty transitions over time, rather than on inter- or multigenerational mobility.                                                                                                                                 1
The paper focuses on how state-level banking deregulation mitigated the adverse effects of the China trade shock on local economies by facilitating consumption smoothing, stabilizing housing prices, and easing the sectoral reallocation of labor.                                                                               1
The paper primarily focuses on the occupational and citizenship transitions of Chinese peasants' children, examining how urbanization and industrialization affect their transition from rural to urban status and the influence of family background and education on these processes.                                             1
The main focus of

In [ ]:
merged_df[(merged_df['Q1'] == 'Yes') & (merged_df['Q2'] == 'Yes') & (merged_df['Q3'] == 'Yes')] #empricial studies: 681

,id,title,year,doi,landing_page,abstract_inverted_index,language,is_oa,oa_status,oa_link,...,len_cited_by,ref_count,cited_by_count,Q1,Q1_1,Q2,Q2_1,Q2_2,Q3,Q4
1,https://openalex.org/W2564827629,The fading American dream: Trends in absolute ...,2017,https://doi.org/10.1126/science.aal4617,https://doi.org/10.1126/science.aal4617,"{'Aspiring': [0], 'to': [1, 18, 34, 42, 47, 10...",en,True,bronze,https://apps.bea.gov/scb/pdf/2007/02%20Februar...,...,702.0,18.0,703.0,Yes,The paper examines intergenerational income mo...,Yes,"The paper refers to 'absolute income mobility,...",N/A,Yes,N/A
44,https://openalex.org/W1966505070,On Square Ordinal Contingency Tables: A Compar...,2009,https://doi.org/10.1111/j.1467-985x.2008.00575.x,https://doi.org/10.1111/j.1467-985x.2008.00575.x,"{'Summary': [0], 'Square': [1], 'contingency':...",en,True,bronze,https://academic.oup.com/jrsssa/article-pdf/17...,...,28.0,13.0,30.0,Yes,The paper compares social class and income mob...,Yes,It describes using the level of association de...,N/A,Yes,N/A
50,https://openalex.org/W3001018177,Association of Social Mobility With the Income...,2020,https://doi.org/10.1001/jamainternmed.2019.6532,https://doi.org/10.1001/jamainternmed.2019.6532,"{'<h3>Importance</h3>': [0], 'Despite': [1], '...",en,True,hybrid,https://jamanetwork.com/journals/jamainternalm...,...,23.0,57.0,23.0,Yes,The paper operationalizes county-level social ...,Yes,The measure mentioned is the association betwe...,N/A,Yes,N/A
54,https://openalex.org/W2120737087,Health-related quality of life and income-rela...,2014,https://doi.org/10.1186/1477-7525-12-52,https://doi.org/10.1186/1477-7525-12-52,"{'To': [0], 'assess': [1], 'the': [2, 9], 'ass...",en,True,gold,https://hqlo.biomedcentral.com/counter/pdf/10....,...,21.0,35.0,21.0,Yes,The paper examines income‐related social mobil...,Yes,The study categorizes participants based on th...,N/A,Yes,N/A
75,https://openalex.org/W3041900243,Trends in Absolute Income Mobility in North Am...,2020,https://doi.org/10.2139/ssrn.3648796,https://doi.org/10.2139/ssrn.3648796,"{'We': [0, 85], 'compute': [1], 'rates': [2, 4...",en,True,green,https://www.econstor.eu/bitstream/10419/223898...,...,12.0,0.0,12.0,Yes,The paper examines absolute upward income mobi...,Yes,The paper employs the measure of absolute upwa...,N/A,Yes,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2157,https://openalex.org/W46480406,Socio-Economic Differentiation of the Populati...,2010,NaN,https://publications.hse.ru/mirror/pubs/share/...,"{'Gordey': [0], 'Yastrebov': [1], '-': [2], 'L...",en,False,closed,NaN,...,1.0,0.0,1.0,Yes,The paper examines intergenerational mobility ...,Yes,It mentions the development of a corresponding...,N/A,Yes,N/A
2160,https://openalex.org/W623596649,Do Education and Sex Matter For Australia’s In...,2012,NaN,https://espace.library.uq.edu.au/view/UQ:291507,"{'The': [0, 130, 190], 'purpose': [1], 'of': [...",en,False,closed,NaN,...,0.0,0.0,0.0,Yes,The paper focuses on intergenerational earning...,Yes,The measures mentioned include intergeneration...,N/A,Yes,N/A
2161,https://openalex.org/W655971325,Estimating Intergenerational Schooling Effects...,2006,NaN,http://www.diva-portal.org/smash/record.jsf?pi...,"{'Estimating': [0, 79], 'Long-Term': [1], 'Con...",en,False,closed,NaN,...,15.0,0.0,15.0,Yes,The paper discusses intergenerational mobility...,Yes,It uses measures such as comparing parental an...,N/A,Yes,N/A
2164,https://openalex.org/W765699370,Local Measures of Intergenerational Mobility o...,2014,https://doi.org/10.2139/ssrn.2496826,https://doi.org/10.2139/ssrn.2496826,"{'Using': [0], 'NLSY': [1], 'data': [2], 'we':...",en,False,closed,NaN,...,0.0,51.0,0.0,Yes,The paper investigates intergenerational incom...,Yes,The paper employs the varying coefficient mode...,N/A,Yes,N/A


In [ ]:
merged_df[(merged_df['Q1'] == 'Yes') & (merged_df['Q2'] == 'Yes') & (merged_df['Q4'] == 'Yes')].Q2_1.values #theoretical studies: 47

array(["The measures mentioned include the elasticity of a child's income with respect to parental educational investment and the evaluation of an optimal progressive income tax rate within a theoretical model.",
       "It considers measures such as differentiated cognitive and non‐cognitive skills, parenting skills, and public education quality as proxies of parents' and children's human capital.",
       'It mentions transition probabilities as a measure to study social mobility.',
       'It mentions relative mobility, which looks at whether individuals are better off than their parents, and absolute mobility, which considers the extent of movement up or down the income ladder.',
       'It refers to statistical models of income dynamics and reviews empirical studies that measure income persistence, though it does not specify detailed indices or numerical measures.',
       'The paper mentions a generalized concentration curve as a tool to study the distribution of prospects, along

In [ ]:
merged_df[(merged_df['Q1'] == 'Yes') & (merged_df['Q2'] == 'No')].Q2_2.values #studies without measures mentioned: 263
# Looks like some theoretical studies might have been missed by Q2.
# But it is quite correct that some studies don't mention any measures or they talk about mobility to study something else (so they might already have a mobility variable).

array(['The provided abstract and details do not explicitly mention any particular measures for studying inter/multigenerational wealth/income/earning mobility.',
       'The information provided (title and citation) does not include any details about specific measures used to study intergenerational income mobility.',
       'The available information does not provide any details on specific measures used to study intergenerational mobility.',
       'The paper appears to focus on a conceptual discussion about the determinants of economic mobility rather than detailing specific quantitative measures for inter/multigenerational mobility.',
       'The paper focuses on a theoretical model of how mobility experiences inform political beliefs rather than proposing or employing specific measures to quantify inter/multigenerational mobility.',
       'The paper does not introduce specific quantitative measures for intergenerational mobility; instead, it focuses on a theoretical model to der

In [ ]:
# merged_df.to_csv(f'{path}/data_abstracts/abstracts_citations_gpt_2025-03-10.csv', index=False)

In [169]:
len(merged_df[(merged_df['Q1'] == 'Yes') & (merged_df['Q2'] == 'Yes')].Q2_1.values) # Only the studies that mention mobility measures (at least in their abstracts or titles).

747

# Getting Extra Abstracts where abstract and abstract_sm are Null. 
💚 I also need all the abstracts (and abstract_sm's for `common_ids`). But some non-NaN cases are just error messages. 💩

In [18]:
data[data['id'].isin(common_ids)][['doi', 'title', 'abstract', 'abstract_sm', 'Q2_1']]

,doi,title,abstract,abstract_sm,Q2_1
18,https://doi.org/10.2307/2648054,Have family income mobility patterns changed?,NaN,We examine the mobility of individuals in the ...,NaN
149,https://doi.org/10.1177/000169936600900107,"Family Background and Income, School Career an...",NaN,to the social mobility chances of youngsters o...,NaN
150,https://doi.org/10.25071/1874-6322.40427,"More Educated, Less Mobile? Diverging Trends i...",NaN,We analyse intergenerational persistence in in...,The study employs measures such as intergenera...
1057,https://doi.org/10.1111/j.1468-0084.1991.mp530...,MEASURING MOBILITY IN WEALTH: SOME ESTIMATES F...,NaN,This paper examines a dynamic aspect of the di...,NaN
1075,https://doi.org/10.25071/1874-6322.23683,"Labor Supply Decisions, Occupational Segregati...",NaN,The article directs attention to the structuri...,The study uses a static labor supply model app...
1083,https://doi.org/10.25071/1874-6322.681,A Comparison of income and Wealth Mobility in ...,NaN,This paper presents a systematic comparison of...,NaN
1144,https://doi.org/10.1016/j.econlet.2012.08.039,Intergenerational income mobility revisited: E...,NaN,Error fetching abstract,The measure mentioned is the income dynamic mo...
1147,https://doi.org/10.25071/1874-6322.23682,Intergenerational Earnings Mobilities – How Se...,NaN,Error fetching abstract,The specific income measures are not detailed ...
1148,https://doi.org/10.1016/j.childyouth.2014.03.002,Intergenerational income mobility and public e...,NaN,Error fetching abstract,The brief information does not specify the pre...
1149,https://doi.org/10.1016/j.chieco.2014.10.005,Intergenerational transfer of human capital an...,NaN,Error fetching abstract,The paper refers to measures of parental human...


In [19]:
# for id in common_ids:
#     data.loc[data['id'] == id, 'abstract'] = data.loc[data['id'] == id, 'abstract_sm'].values[0]

In [20]:
import requests

# Extract DOI from DOI URL
def extract_doi(doi_url):
    return doi_url.split("doi.org/")[-1]

# Semantic Scholar API offers more abstracts. So do sementic scholar first and then crossref.
def get_semantic_scholar_data(doi):
    doi_short = extract_doi(doi)
    url = f"https://api.semanticscholar.org/v1/paper/{doi_short}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        abstract = data.get("abstract", "No abstract available")
        authors = [author["name"] for author in data.get("authors", [])] # It's easier to extract authors from Semantic Scholar.
        return abstract, authors
    else:
        return "Error fetching abstract", []  # Ensure it always returns a tuple


def main_semantic_scholar(df):
    print(f"Extracting abstracts for {len(df)} papers")
    count_fail = 0
    start = time.time()
    df['abstract_sm'] = None
    df['authors_sm'] = None

    for doi in df['doi']: #Already exisitng doi's
        print(f"Extracting abstract for {doi}")
        if pd.isna(doi) == False:
            abstract, authors = get_semantic_scholar_data(doi)
            df.loc[df['doi'] == doi, 'abstract_sm'] = abstract
            df.loc[df['doi'] == doi, 'authors_sm'] = "; ".join(authors)
        else:
            count_fail += 1

    print(f"\nAbstracts extracted for {len(df) - count_fail} papers. {count_fail} papers failed to extract abstracts. \nTime taken: {time.time() - start} seconds")


In [ ]:
# for row in data[data['id'].isin(common_ids) & data['abstract_sm'].str.contains('Error fetching abstract', na=False)].itertuples(index=False):
#     doi = row.doi
#     abstract, authors = get_semantic_scholar_data(doi)
#     data.loc[data['doi'] == doi, 'abstract_sm'] = abstract
#     data.loc[data['doi'] == doi, 'authors_sm'] = "; ".join(authors)

### Try with Elsevier API

In [21]:
len(data[data['id'].isin(common_ids) & data['abstract_sm'].str.contains('Error fetching abstract', na=False)])

42

In [22]:
from elsapy.elsclient import ElsClient
from elsapy.elsdoc import AbsDoc
from elsapy.elssearch import ElsSearch

API_KEY = "090ced1fb838c51870bd6d5d04220727"
client = ElsClient(API_KEY)

for row in data[data['id'].isin(common_ids) & data['abstract_sm'].str.contains('Error fetching abstract', na=False)].itertuples(index=False):
    print(f"Extracting abstract for {row.doi}")
    doi = extract_doi(row.doi)
    doc_srch = ElsSearch(f"DOI({doi})", 'scopus')
    doc_srch.execute(client, get_all = False)
    if len(doc_srch.results) > 1:
        print(f"Multiple results found for {doi}.")
    elif len(doc_srch.results) == 0:
        print(f"No results found for {doi}.")
    else:
        result = doc_srch.results[0]
        if 'error' not in result:
            scopus_id = result['dc:identifier'].split(":")[-1]
            scp_doc = AbsDoc(scp_id = scopus_id)

            if scp_doc.read(client):
                coredata = scp_doc.data.get('coredata', {})  # sometimes missing
                if 'dc:description' in coredata:
                    data.loc[data['doi'] == row.doi, 'abstract_sm'] = coredata['dc:description']
            else:
                print ("Read document failed.")

        else:
            print(f"No identifier found for {doi}.")

    

Extracting abstract for https://doi.org/10.1016/j.econlet.2012.08.039
Extracting abstract for https://doi.org/10.25071/1874-6322.23682
No identifier found for 10.25071/1874-6322.23682.
Extracting abstract for https://doi.org/10.1016/j.childyouth.2014.03.002
Extracting abstract for https://doi.org/10.1016/j.chieco.2014.10.005
Extracting abstract for https://doi.org/10.1007/s11516-008-0002-x
Extracting abstract for https://doi.org/10.1016/j.econlet.2010.05.018
Extracting abstract for https://doi.org/10.1016/j.jpolmod.2009.07.004
Extracting abstract for https://doi.org/10.1016/s0165-1765(03)00084-3
Extracting abstract for https://doi.org/10.1007/s10888-005-9015-3
Extracting abstract for https://doi.org/10.1111/j.1467-6435.1976.tb01992.x
Extracting abstract for https://doi.org/10.1007/s10888-017-9372-8
Extracting abstract for https://doi.org/10.1016/j.chieco.2015.12.009
Extracting abstract for https://doi.org/10.1007/s12552-016-9161-1
Extracting abstract for https://doi.org/10.1007/s13524-

In [24]:
len(data[data['id'].isin(common_ids) & data['abstract_sm'].str.contains('Error fetching abstract', na=False)])

11

In [25]:
# We should be able to extract the abstracts of the SSRN papers,,. We'll use Semantic Scholar for this and also for the others that didn't manage to get abstracts from Elsevier.

for row in data[data['id'].isin(common_ids) & data['abstract_sm'].str.contains('Error fetching abstract', na=False)].itertuples(index=False):
    print(f"Extracting abstract for {row.doi}")
    doi = row.doi
    abstract, authors = get_semantic_scholar_data(doi)
    data.loc[data['doi'] == doi, 'abstract_sm'] = abstract
    data.loc[data['doi'] == doi, 'authors_sm'] = "; ".join(authors)
print('✅ Done!')

Extracting abstract for https://doi.org/10.25071/1874-6322.23682
Extracting abstract for https://doi.org/10.1111/j.1467-6435.1976.tb01992.x
Extracting abstract for https://doi.org/10.1177/0973703020130202
Extracting abstract for https://doi.org/10.17053/jcc.2017.20.3.008
Extracting abstract for https://doi.org/10.14288/1.0395499
Extracting abstract for https://doi.org/10.17654/0972361722034
Extracting abstract for https://doi.org/10.2139/ssrn.5009798
Extracting abstract for https://doi.org/10.2139/ssrn.5061087
Extracting abstract for https://doi.org/10.2139/ssrn.5095005
Extracting abstract for https://doi.org/10.25003/ras.13.01.0006
Extracting abstract for https://doi.org/10.12677/sd.2025.151027
✅ Done!


In [33]:
data[data['id'].isin(common_ids) & data['abstract_sm'].isna()][['doi', 'abstract_sm']] # These 8 out of 11 papers don't have abstracts in Semantic Scholar.

,doi,abstract_sm
1299,https://doi.org/10.1111/j.1467-6435.1976.tb019...,None
1525,https://doi.org/10.17053/jcc.2017.20.3.008,None
1682,https://doi.org/10.14288/1.0395499,None
1899,https://doi.org/10.17654/0972361722034,None
2140,https://doi.org/10.2139/ssrn.5061087,None
2150,https://doi.org/10.2139/ssrn.5095005,None
2153,https://doi.org/10.25003/ras.13.01.0006,None
2155,https://doi.org/10.12677/sd.2025.151027,None


In [35]:
from crossref.restful import Works
works = Works()


count_fail = 0
start = time.time()
for row in data[data['id'].isin(common_ids) & data['abstract_sm'].isna()].itertuples(index=False):
    print(f"Retry: {row.doi} with crossref")
    result = works.doi(row.doi)
    if result is not None and "abstract" in result.keys():
        data.loc[data['doi'] == row.doi, 'abstract_sm'] = result['abstract']
        print(result['abstract'])
    else:
        count_fail += 1
        data.loc[data['doi'] == row.doi, 'abstract_sm'] = None

print(f"\n{count_fail} doi's with no abstract available. Time taken: {time.time() - start} seconds") # None had extra info on Crossref.           

Retry: https://doi.org/10.1111/j.1467-6435.1976.tb01992.x with crossref
Retry: https://doi.org/10.17053/jcc.2017.20.3.008 with crossref
Retry: https://doi.org/10.14288/1.0395499 with crossref
Retry: https://doi.org/10.17654/0972361722034 with crossref
Retry: https://doi.org/10.2139/ssrn.5061087 with crossref
Retry: https://doi.org/10.2139/ssrn.5095005 with crossref
Retry: https://doi.org/10.25003/ras.13.01.0006 with crossref
Retry: https://doi.org/10.12677/sd.2025.151027 with crossref

8 doi's with no abstract available. Time taken: 3.1936609745025635 seconds


In [39]:
data = data[~(data['id'].isin(common_ids) & data['abstract_sm'].isna())] # Remove the ones that still don't have abstracts (in abstract nor in abstract_sm)

In [40]:
data.to_csv(f'{path}/data_abstracts/abstracts_citations_gpt_extra_{today}.csv', index=False)

Now, we have to try GPT again and get Q1-4 again. Because we only realized this issue today.
Then we're going to merge the dataframe again (i.e. update the dataframe).
Then, we will get more properties and relationship types.
Then, create the network.